In [1]:
# -*- coding: utf-8
# VOLT DATA LAB (https://www.voltdata.info/)
# Reinaldo Chaves (@paidatocandeira)
# Mostra quais projetos de lei tiveram aprovação e as falhas de transparência do sistema 
#

In [2]:
import json
import xmltodict
import pandas as pd
import numpy as np
from datetime import date

# Tramitações

# O sistema de dados abertos da Alesp tem um arquivo enorme de histórico de tramitações

https://www.al.sp.gov.br/dados-abertos/recurso/101

# Mas esse arquivo tem erros de formatação nos anos mais antigos e não abre

Há uma alternativa, um arquivo menor e bem formatado, mas ele cobre só até 09/2017

Atual

https://www.al.sp.gov.br/dados-abertos/recurso/221

In [3]:
with open("dados/tramitacao/documento_andamento_atual_set_2018.xml", 'r') as f:
    xmlString = f.read()

In [4]:
jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)

In [5]:
with open("output.json", 'w') as f:
    f.write(jsonString)

In [6]:
arquivo_json = open('output.json', 'r')
dados_json = json.load(arquivo_json)

In [7]:
#dados_json['documentos_autores']['DocumentoAutor']

In [8]:
lista = dados_json['documentos_andamentos']['DocumentoAndamento']

In [9]:
df_andamentos = pd.DataFrame(lista, columns = ['Descricao', 
                                               'Data', 
                                               'IdComissao', 
                                               'IdDocumento', 
                                               'IdEtapa', 
                                               'IdTpAndamento', 
                                               'NmEtapa', 
                                               'NrOrdem', 
                                               'TpAndamento'])

In [13]:
df_andamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224514 entries, 0 to 224513
Data columns (total 9 columns):
Descricao        224513 non-null object
Data             224514 non-null object
IdComissao       224514 non-null object
IdDocumento      224514 non-null object
IdEtapa          224514 non-null object
IdTpAndamento    224514 non-null object
NmEtapa          224514 non-null object
NrOrdem          224514 non-null object
TpAndamento      224514 non-null object
dtypes: object(9)
memory usage: 15.4+ MB


In [14]:
df_andamentos.reset_index().head()

,index,Descricao,Data,IdComissao,IdDocumento,IdEtapa,IdTpAndamento,NmEtapa,NrOrdem,TpAndamento
0,0,Arquivado pelo Setor de Arquivo na caixa 13.01...,2018-06-06T00:00:00-03:00,0,3239,17,32,Arquivo,0,E
1,1,Enviado de SERVIÇO DE ARQUIVO para DDI-DEP. DE...,2017-10-31T00:00:00-02:00,0,4389,32,53,Não Identificada,0,I
2,2,Recebido por DDI-DEP. DE DOCUM. INFORMAÇÃO (Gu...,2017-11-24T00:00:00-02:00,0,4389,32,53,Não Identificada,0,I
3,3,Enviado de DDI-DEP. DE DOCUM. INFORMAÇÃO para ...,2017-11-24T00:00:00-02:00,0,4389,32,53,Não Identificada,1,I
4,4,Recebido por SERVIÇO DE ARQUIVO (Guia 15001/2017),2017-12-21T00:00:00-02:00,0,4389,32,53,Não Identificada,0,I


In [12]:
#a = df_andamentos['Descricao'].value_counts()

# Filtra só as linhas que têm alguma aprovação de projeto de lei

In [18]:
mask = df_andamentos['Descricao'].str.contains("Aprovado o Projeto", na=False)
aprovado = df_andamentos[mask]

In [19]:
aprovado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 1217 to 196828
Data columns (total 9 columns):
Descricao        264 non-null object
Data             264 non-null object
IdComissao       264 non-null object
IdDocumento      264 non-null object
IdEtapa          264 non-null object
IdTpAndamento    264 non-null object
NmEtapa          264 non-null object
NrOrdem          264 non-null object
TpAndamento      264 non-null object
dtypes: object(9)
memory usage: 20.6+ KB


In [20]:
aprovado.to_excel('resultados/projetos_aprovados.xlsx',sheet_name='Sheet1')

# Carrega a base de palavras-chave/projetos criada no notebook alesp_projetos

In [21]:
palavras_chave_dep = pd.read_excel("resultados/palavras_chave_projetos_deputados.xlsx", sheet_name='Sheet1', converters={'IdDocumento': lambda x: str(x), 
                                                                                                                         'IdPalavra': lambda x: str(x), 
                                                                                                                                 'IdAutor': lambda x: str(x), 
                                                                                                                         'CodOriginalidade': lambda x: str(x), 
                                                                                                                         'IdNatureza': lambda x: str(x), 
                                                                                                                         'NroLegislativo': lambda x: str(x),
                                                                                                                         'codigo_unico': lambda x: str(x),
                                                                                                                         'sequencial': lambda x: str(x),
                                                                                                                         'cpf': lambda x: str(x)
                                                                                                                         })

In [22]:
palavras_chave_dep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31869 entries, 0 to 31868
Data columns (total 22 columns):
IdAutor             31869 non-null object
IdDocumento         31869 non-null object
NomeAutor           31869 non-null object
codigo_unico        31869 non-null object
nome_deputado       31869 non-null object
uf                  31869 non-null object
nome_completo       31869 non-null object
sequencial          31869 non-null object
cpf                 31869 non-null object
nome_urna           31869 non-null object
partido_eleicao     31869 non-null object
situacao            31869 non-null object
AnoLegislativo      31869 non-null int64
CodOriginalidade    24430 non-null object
DtEntradaSistema    31869 non-null datetime64[ns]
DtPublicacao        31869 non-null datetime64[ns]
Ementa              31869 non-null object
IdNatureza          31869 non-null object
NroLegislativo      31869 non-null object
IdPalavra           31869 non-null object
Palavra             31869 non-null o

In [23]:
projetos_aprovados = pd.merge(palavras_chave_dep, aprovado, left_on='IdDocumento', right_on='IdDocumento')

In [24]:
projetos_aprovados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6308 entries, 0 to 6307
Data columns (total 30 columns):
IdAutor             6308 non-null object
IdDocumento         6308 non-null object
NomeAutor           6308 non-null object
codigo_unico        6308 non-null object
nome_deputado       6308 non-null object
uf                  6308 non-null object
nome_completo       6308 non-null object
sequencial          6308 non-null object
cpf                 6308 non-null object
nome_urna           6308 non-null object
partido_eleicao     6308 non-null object
situacao            6308 non-null object
AnoLegislativo      6308 non-null int64
CodOriginalidade    864 non-null object
DtEntradaSistema    6308 non-null datetime64[ns]
DtPublicacao        6308 non-null datetime64[ns]
Ementa              6308 non-null object
IdNatureza          6308 non-null object
NroLegislativo      6308 non-null object
IdPalavra           6308 non-null object
Palavra             6308 non-null object
PalavraSemAcento  

In [25]:
len(projetos_aprovados['NomeAutor'].value_counts())

95

In [26]:
a = len(projetos_aprovados['IdDocumento'].value_counts())
a

195

Deputados com projetos aprovados

In [27]:
projetos_aprovados.groupby('NomeAutor')['IdDocumento'].nunique().sort_values(ascending=False)

NomeAutor
Sebastião Santos                     10
Pedro Tobias                          7
Carlão Pignatari                      7
Edson Giriboni                        7
Caio França                           7
Márcio Camargo                        7
Afonso Lobato                         6
Itamar Borges                         6
Roberto Engler                        6
Welson Gasparini                      5
Fernando Cury                         5
Chico Sardelli                        5
Luiz Carlos Gondim                    5
Marcos Damasio                        5
Mauro Bragato                         5
Paulo Correa Jr                       5
Maria Lúcia Amary                     5
Roque Barbiere                        5
Roberto Massafera                     4
Rita Passos                           4
Luiz Fernando T. Ferreira             4
Wellington Moura                      4
Célia Leão                            4
Davi Zaia                             4
João Paulo Rillo              

Palavras-chave dos projetos aprovados

In [28]:
projetos_aprovados.groupby('Palavra')['IdDocumento'].nunique().sort_values(ascending=False)

Palavra
DENOMINAÇÃO                               64
TURISMO                                   28
LAZER                                     24
DESENVOLVIMENTO ECONÔMICO E SOCIAL        24
ESTÂNCIA TURÍSTICA                        24
DESENVOLVIMENTO TURÍSTICO                 24
VIADUTO                                   18
RODOVIA                                   13
ESCOLA                                    12
MUNICÍPIO DE INTERESSE TURÍSTICO          11
PASSARELA                                  8
PONTE                                      7
PROIBIÇÃO                                  5
INTERESSE TURÍSTICO                        5
BARRETOS (MUNICÍPIO)                       4
CONSCIENTIZAÇÃO                            4
DISPOSITIVO DE ACESSO                      4
MOGI DAS CRUZES (MUNICÍPIO)                4
TREVO                                      4
COMPLEXO VIÁRIO                            3
VOTUPORANGA (MUNICÍPIO)                    3
CRUZEIRO (MUNICÍPIO)                       3
ES

In [29]:
projetos_aprovados.to_excel('resultados/projetos_aprovados_deputados_selecioanados.xlsx',sheet_name='Sheet1')